[View in Colaboratory](https://colab.research.google.com/github/josd/eye/blob/master/transduction/transduction_bodies/observation_prediction_bodies.ipynb)

# Transduction from observation to prediction for bodies

## Introduction

What is [Transduction (machine learning)](https://en.wikipedia.org/wiki/Transduction_(machine_learning%29):

> In logic, statistical inference, and supervised learning, transduction or
transductive inference is reasoning from observed, specific (training) cases
to specific (test) cases. In contrast, induction is reasoning from observed
training cases to general rules, which are then applied to the test cases.
The distinction is most interesting in cases where the predictions of the
transductive model are not achievable by any inductive model. Note that this
is caused by transductive inference on different test sets producing mutually
inconsistent predictions.

What is the Tensor2Tensor [Transformer model](https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/models/transformer.py):

> The Transformer model consists of an encoder and a decoder. Both are stacks
of self-attention layers followed by feed-forward layers. This model yields
good results on a number of problems, especially in NLP and machine translation.
See "Attention Is All You Need" (https://arxiv.org/abs/1706.03762) for the full
description of the model and the results obtained with its early version.

![Transformer model](https://pbs.twimg.com/media/DCKhefrUMAE9stK.jpg)

> The encoder is composed of a stack of N identical layers. Each layer has
two sub-layers. The first is a multi-head self-attention mechanism, and the
second is a simple, positionwise fully connected feed-forward network.
There is a residual connection around each of the two sub-layers, followed by
layer normalization.

> The decoder is also composed of a stack of N identical layers. In addition
to the two sub-layers in each encoder layer, the decoder inserts a third
sub-layer, which performs multi-head attention over the output of the encoder
stack. The self-attention sub-layer in the decoder stack is modified to prevent
positions from attending to subsequent positions.  This masking, combined with
the fact that the output embeddings are offset by one position, ensures that the
predictions for position i can depend only on the known outputs at positions
less than i.

In [1]:
# Preparation

! pip install -U tensor2tensor
! curl -o observation_prediction_bodies.sh http://josd.github.io/eye/transduction/transduction_bodies/observation_prediction_bodies.sh
! curl -o observation_prediction_bodies.py http://josd.github.io/eye/transduction/transduction_bodies/observation_prediction_bodies.py
! curl -o __init__.py http://josd.github.io/eye/transduction/transduction_bodies/__init__.py
! curl -o sample_bodies.observation http://josd.github.io/eye/transduction/transduction_bodies/sample_bodies.observation
! chmod +x observation_prediction_bodies.sh
% rm -fr ~/t2t_data/observation_prediction_bodies
% rm -fr ~/t2t_train/observation_prediction_bodies/transformer-transformer_small/

Requirement already up-to-date: tensor2tensor in /usr/local/lib/python3.6/dist-packages (1.6.2)
Requirement not upgraded as not directly required: gevent in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (1.3.0)
Requirement not upgraded as not directly required: flask in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (1.0.2)
Requirement not upgraded as not directly required: gym in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (0.10.5)
Requirement not upgraded as not directly required: h5py in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (2.7.1)
Requirement not upgraded as not directly required: gunicorn in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (19.8.1)
Requirement not upgraded as not directly required: scipy in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (0.19.1)
Requirement not upgraded as not directly required: six in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (1.11.0)
Requi

In [2]:
# See the observation_prediction_bodies problem

! pygmentize -g observation_prediction_bodies.py

import random
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.utils import registry

@registry.register_problem
class ObservationPredictionBodies(text_problems.Text2TextProblem):
  """Transduction from observation to prediction for bodies."""

  @property
  def approx_vocab_size(self):
    return 2**14  # ~16k

  @property
  def is_generate_per_split(self):
    # generate_data will shard the data into TRAIN and EVAL for us.
    return False

  @property
  def dataset_splits(self):
    """Splits of data to produce and number of output shards for each."""
    return [{
        "split": problem.DatasetSplit.TRAIN,
        "shards": 7,
    }, {
        "split": problem.DatasetSplit.EVAL,
        "shards": 3,
    }]

  def generate_samples(self, data_dir, tmp_dir, dataset_split):
    del data_dir
    del tmp_dir
    del dataset_split

    for n in range(100000):
      # wind turbine size factor
      size_factor = 

In [3]:
# See the observation_prediction_bodies script

! pygmentize -g observation_prediction_bodies.sh

#!/bin/bash
PROBLEM=observation_prediction_bodies
MODEL=transformer
HPARAMS=transformer_small

USER_DIR=$PWD
DATA_DIR=$HOME/t2t_data/$PROBLEM
TMP_DIR=/tmp/t2t_datagen/$PROBLEM
TRAIN_DIR=$HOME/t2t_train/$PROBLEM/$MODEL-$HPARAMS

mkdir -p $DATA_DIR $TMP_DIR $TRAIN_DIR

# Generate data
t2t-datagen \
  --data_dir=$DATA_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --tmp_dir=$TMP_DIR

# Train
t2t-trainer \
  --data_dir=$DATA_DIR \
  --eval_steps=10 \
  --hparams_set=$HPARAMS \
  --local_eval_frequency=100 \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --train_steps=2000 \
  --worker_gpu_memory_fraction=0.75

# Decode
t2t-decoder \
  --data_dir=$DATA_DIR \
  --decode_from_file=sample_bodies.observation \
  --decode_hparams="beam_size=3,alpha=0.6,return_beams=True,write_beam_scores=True" \
  --decode_to_file=sample_bodies.prediction \
  --hparams_set=$HPARAMS \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBL

In [4]:
# Run the observation_prediction_bodies script

! ./observation_prediction_bodies.sh

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Importing user module content from path /
INFO:tensorflow:Generating problems:
    observation:
      * observation_prediction_bodies
INFO:tensorflow:Generating data for observation_prediction_bodies.
INFO:tensorflow:Generating vocab file: /content/t2t_data/observation_prediction_bodies/vocab.observation_prediction_bodies.16384.subwords
INFO:tensorflow:Trying min_count 500
INFO:tensorflow:Iteration 0
INFO:tensorflow:vocab_size = 431
INFO:tensorflow:Iteration 1
INFO:tensorflow:vocab_size = 324
INFO:tensorflow:Iteration 2
INFO:tensorflow:vocab_size = 329
INFO:tensorflow:Iteration 3
INFO:tensorflow:vocab_size = 327
INFO:tensorflow:Trying min_count 250
INFO:tensorflow:Iterati

INFO:tensorflow:Base learning rate: 0.200000
INFO:tensorflow:Trainable Variables Total size: 3912960
INFO:tensorflow:Using optimizer Adam
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
2018-05-13 21:33:27.910063: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:898] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-05-13 21:33:27.910527: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1344] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
totalMemory: 11.17GiB freeMemory: 11.

INFO:tensorflow:Training model for 100 steps
INFO:tensorflow:Reading data files from /content/t2t_data/observation_prediction_bodies/observation_prediction_bodies-train*
INFO:tensorflow:partition: 0 num_data_files: 7
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Setting T2TModel mode to 'train'
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_873_256.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_873_256.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_873_256.top
INFO:tensorflow:Base learning rate: 0.200000
INFO:tensorflow:Trainable Variables Total size: 3912960
INFO:tensorflow:Using optimizer Adam
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
2018-05-13 21:34:19.262084: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gp

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Setting T2TModel mode to 'train'
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_873_256.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_873_256.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_873_256.top
INFO:tensorflow:Base learning rate: 0.200000
INFO:tensorflow:Trainable Variables Total size: 3912960
INFO:tensorflow:Using optimizer Adam
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
2018-05-13 21:35:09.876651: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gpu devices: 0
2018-05-13 21:35:09.876761: I tensorflow/core/common_runtime/gpu/gpu_device.cc:911] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-05-13 21:35:09.876796: I tensorflow/core/common_run

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Setting T2TModel mode to 'train'
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_873_256.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_873_256.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_873_256.top
INFO:tensorflow:Base learning rate: 0.200000
INFO:tensorflow:Trainable Variables Total size: 3912960
INFO:tensorflow:Using optimizer Adam
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
2018-05-13 21:36:00.516498: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gpu devices: 0
2018-05-13 21:36:00.516613: I tensorflow/core/common_runtime/gpu/gpu_device.cc:911] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-05-13 21:36:00.516645: I tensorflow/core/common_run

...

INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_873_256.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_873_256.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_873_256.top
INFO:tensorflow:Base learning rate: 0.200000
INFO:tensorflow:Trainable Variables Total size: 3912960
INFO:tensorflow:Using optimizer Adam
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
2018-05-13 21:47:08.395189: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gpu devices: 0
2018-05-13 21:47:08.395303: I tensorflow/core/common_runtime/gpu/gpu_device.cc:911] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-05-13 21:47:08.395351: I tensorflow/core/common_runtime/gpu/gpu_device.cc:917]      0 
2018-05-13 21:47:08.395374: I tensorflow/core/common_runtime/gpu/gpu_device.cc:930] 0:   N 
2018-05-13 21:47:08.

INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_873_256.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_873_256.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_873_256.top
INFO:tensorflow:Base learning rate: 0.200000
INFO:tensorflow:Trainable Variables Total size: 3912960
INFO:tensorflow:Using optimizer Adam
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
2018-05-13 21:47:59.601440: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gpu devices: 0
2018-05-13 21:47:59.601563: I tensorflow/core/common_runtime/gpu/gpu_device.cc:911] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-05-13 21:47:59.601596: I tensorflow/core/common_runtime/gpu/gpu_device.cc:917]      0 
2018-05-13 21:47:59.601620: I tensorflow/core/common_runtime/gpu/gpu_device.cc:930] 0:   N 
2018-05-13 21:47:59.

INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_873_256.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_873_256.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_873_256.top
INFO:tensorflow:Base learning rate: 0.200000
INFO:tensorflow:Trainable Variables Total size: 3912960
INFO:tensorflow:Using optimizer Adam
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
2018-05-13 21:48:50.641750: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gpu devices: 0
2018-05-13 21:48:50.641871: I tensorflow/core/common_runtime/gpu/gpu_device.cc:911] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-05-13 21:48:50.641913: I tensorflow/core/common_runtime/gpu/gpu_device.cc:917]      0 
2018-05-13 21:48:50.641941: I tensorflow/core/common_runtime/gpu/gpu_device.cc:930] 0:   N 
2018-05-13 21:48:50.

INFO:tensorflow:Setting T2TModel mode to 'train'
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_873_256.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_873_256.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_873_256.top
INFO:tensorflow:Base learning rate: 0.200000
INFO:tensorflow:Trainable Variables Total size: 3912960
INFO:tensorflow:Using optimizer Adam
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
2018-05-13 21:49:41.631080: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gpu devices: 0
2018-05-13 21:49:41.631204: I tensorflow/core/common_runtime/gpu/gpu_device.cc:911] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-05-13 21:49:41.631240: I tensorflow/core/common_runtime/gpu/gpu_device.cc:917]      0

INFO:tensorflow:Importing user module content from path /
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1deb145d68>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': 1234, '_save_summary_steps': 100, '_save_checkpoints_secs': None, '_log_step_count_steps': 100, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.75
}
allow_soft_placement: true
graph_options {
  optimiz

In [5]:
# See the transductions
# For each observation the top 3 predictions are shown with their respective log probability

! pygmentize -g sample_bodies.observation
print("->-")
! pygmentize -g sample_bodies.prediction

A_PERSON with weight 74 kg and height 179 cm
A_TURBINE with size factor 4 and subjected to windspeed 62 km/h
->-
A_PERSON has BMI class N	-0.34	A_PERSON has BMI class U	-3.86	A_PERSON has BMI class O	-4.59
A_TURBINE producing 9533 kW	-0.66	A_TURBINE producing 14299 kW	-2.76	A_TURBINE producing 7149 kW	-2.84
